In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import math
from variables import *
import time
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
def getCarLinks(main_soup):
    """ Will generate a list of pages, then loop over each page getting 
    all the car links appending them to a list"""
    links_list = []
    pages = getPageList(main_soup) # Will retrieve a list of pages to loop over it.
    for page in pages:
        web_c = requests.get(page).content 
        soup = BeautifulSoup(web_c, "html.parser")
        links = soup.find_all("a",linkRef)
        for link in links:
            links_list.append(link.get('href'))
    return links_list

def getPageList(soup):
    """ Will take the total car results from the webpage, will calculate how many 
    pages there are, then will create the links to the pages."""
    pages = []
    page_link = soup.find_all("a",pageLink)[0].get("href")
    q_res = soup.find_all("span",results)[0].text
    q_res = q_res.replace(" resultados","")
    q_res = q_res.replace(".","")
    q_res = int(q_res)
    n_pag = math.floor(q_res/48)
    j = 49 # 2nd page
    pages.append(page_link)
    pages.append("{}_Desde_{}".format(page_link, j))
    for i in range(1, n_pag):
        j += 48
        pages.append("{}_Desde_{}".format(page_link, j))
    return pages

In [3]:
def cleanText(text):
    """ Some text fields have different characters we don't need, this function
    will take them out"""
    cl_text = text
    chars = ["\n","\t","Publicado el ",'\"',")","("]
    for char in chars:
        cl_text = cl_text.replace(char,"")
    return cl_text

def getRep(soup):
    """ Sometimes the seller doesn't have reputation, so it will try to retrieve
    this info if it does exist"""
    try:
        rep = clean_text(soup.find("div","reputation-info block").span.text) 
    except:
        rep = soup.find("div","reputation-info block").p.text
    return rep

def getSpecList(soup):
    spec_list = soup.find('ul','specs-list').find_all("li")
    specs = {}
    for spec in spec_list:
        specs[spec.strong.text] = spec.span.text
    specs["Kilómetros"] = int(specs["Kilómetros"].replace(" km","").replace(".",""))
    specs["Año"] = int(specs["Año"])
    return specs

def extraAtrib(link):
    """ There was written function in the code that also had some interesting data,
    I figured out what the parameters were, so I took them and I gave them a new
    description"""
    test_list = str(link.find_all("script")[1]).replace('meli_ga("set", ',"").split(";")[6:-17]
    dims = {"dimension35":"Seller Type","dimension72":"Reputation Code"
            ,"dimension111": ["Brand", "Model"],"dimension49": "Codigo"
            ,"dimension139": "Loc1","dimension140": "Loc2","dimension141": "Loc3"}
    atr = {}
    for elem in test_list:
        for dim,val in dims.items():
            if dim in elem:
                if dim == "dimension111":
                    elem = elem.split(",")
                    atr[val[0]] = cleanText(elem[1].replace("BRAND:","").strip())
                    atr[val[1]] = cleanText(elem[2].replace("MODEL:","").strip())
                else:
                    elem = elem.split(",")
                    atr[val]= cleanText(elem[1]).strip()
    return atr 

In [4]:
def getCarData(link):
    """ 
        Table will contain:
           - Desc
           - Publish Date 
           - Price 
           - Currency 
           - Loc 1 
           - Loc 2 
           - Loc 3 
           - Seller Reputation
           - Año
           - Color
           - Tipo de Combustible
           - Puertas
           - Transmision
           - Motor
           - Tipo de Carroceria
           - Kilometros
           - Link
           - Pulling Date """
    link_req = requests.get(link).content
    temp_soup = BeautifulSoup(link_req)
    car_specs = {}
    car_specs["Desc"] = cleanText(temp_soup.find_all("header","item-title")[0].text)
    car_specs["Publish Date"] = cleanText(temp_soup.find("article","vip-classified-info item-published-date").text)
    car_specs["Currency"] = temp_soup.find_all("fieldset", "item-price")[0].find_all("span", "price-tag-symbol")[0].text
    car_specs["Price"] = int(temp_soup.find_all("fieldset", "item-price")[0].find_all("span", "price-tag-fraction")[0].text.replace(".",""))
    car_specs["Link"] = link
    specs = getSpecList(temp_soup)
    extraSpecs = extraAtrib(temp_soup)
    car_specs.update(specs)
    car_specs.update(extraSpecs)
    return car_specs

In [5]:
carCont = requests.get('https://autos.mercadolibre.com.ar/renault/clio/').content
main_soup = BeautifulSoup(carCont, "html.parser")
carList = getCarLinks(main_soup)

In [6]:
car1 = getCarData(carList[0])

In [7]:
car1

{'Desc': 'Renault Clio 1.2 Campus Pack I 75cv',
 'Publish Date': '22/12/2020',
 'Currency': '$',
 'Price': 600000,
 'Link': 'https://auto.mercadolibre.com.ar/MLA-901952153-renault-clio-12-campus-pack-i-75cv-88973-cp-_JM#position=1&type=item&tracking_id=0ebfc90e-2f22-45fd-abb8-de2b34898b1a',
 'Año': 2012,
 'Color': 'Gris',
 'Tipo de combustible': 'Nafta',
 'Puertas': '3',
 'Transmisión': 'Manual',
 'Motor': '1.2',
 'Tipo de carrocería': 'Hatchback',
 'Kilómetros': 56000,
 'Codigo': 'MLA901952153',
 'Seller Type': 'car_dealer',
 'Reputation Code': '5_green',
 'Brand': 'Renault',
 'Model': 'Clio',
 'Loc1': 'Capital Federal',
 'Loc2': 'Capital Federal',
 'Loc3': 'Palermo'}

In [14]:
%%time

for link in car_list:
    car = getCarData(link)
    carSpecsList.append(car)

CPU times: user 4min 19s, sys: 3.64 s, total: 4min 23s
Wall time: 30min 41s


In [16]:
cars_df = pd.DataFrame(car_specs_list)

#####  Main data Pulling

In [ ]:
%%time
carSpecsList = []
for carMod in carModels:
    carCont = requests.get(carMod).content
    main_soup = BeautifulSoup(carCont, "html.parser")
    carList = getCarLinks(main_soup)
    print(f'Pulling {carMod}')
    q = len(carList)
    i = 1
    for link in carList:
        car = getCarData(link)
        carSpecsList.append(car)
        print(f'Pulled {i} of {q} records')
        i += 1
    print('---------------------------------------------------------------------------')
    if carMod == carModels[-1]:
        print("process Finished")
    else:
        time.sleep(1300)
cars_df = pd.DataFrame(carSpecsList)

Pulling https://autos.mercadolibre.com.ar/renault/clio/
Pulled 1 of 1406 records
Pulled 2 of 1406 records
Pulled 3 of 1406 records
Pulled 4 of 1406 records
Pulled 5 of 1406 records
Pulled 6 of 1406 records
Pulled 7 of 1406 records
Pulled 8 of 1406 records
Pulled 9 of 1406 records
Pulled 10 of 1406 records
Pulled 11 of 1406 records
Pulled 12 of 1406 records
Pulled 13 of 1406 records
Pulled 14 of 1406 records
Pulled 15 of 1406 records
Pulled 16 of 1406 records
Pulled 17 of 1406 records
Pulled 18 of 1406 records
Pulled 19 of 1406 records
Pulled 20 of 1406 records
Pulled 21 of 1406 records
Pulled 22 of 1406 records
Pulled 23 of 1406 records
Pulled 24 of 1406 records
Pulled 25 of 1406 records
Pulled 26 of 1406 records
Pulled 27 of 1406 records
Pulled 28 of 1406 records
Pulled 29 of 1406 records
Pulled 30 of 1406 records
Pulled 31 of 1406 records
Pulled 32 of 1406 records
Pulled 33 of 1406 records
Pulled 34 of 1406 records
Pulled 35 of 1406 records
Pulled 36 of 1406 records
Pulled 37 of 1406

Pulled 307 of 1406 records
Pulled 308 of 1406 records
Pulled 309 of 1406 records
Pulled 310 of 1406 records
Pulled 311 of 1406 records
Pulled 312 of 1406 records
Pulled 313 of 1406 records
Pulled 314 of 1406 records
Pulled 315 of 1406 records
Pulled 316 of 1406 records
Pulled 317 of 1406 records
Pulled 318 of 1406 records
Pulled 319 of 1406 records
Pulled 320 of 1406 records
Pulled 321 of 1406 records
Pulled 322 of 1406 records
Pulled 323 of 1406 records
Pulled 324 of 1406 records
Pulled 325 of 1406 records
Pulled 326 of 1406 records
Pulled 327 of 1406 records
Pulled 328 of 1406 records
Pulled 329 of 1406 records
Pulled 330 of 1406 records
Pulled 331 of 1406 records
Pulled 332 of 1406 records
Pulled 333 of 1406 records
Pulled 334 of 1406 records
Pulled 335 of 1406 records
Pulled 336 of 1406 records
Pulled 337 of 1406 records
Pulled 338 of 1406 records
Pulled 339 of 1406 records
Pulled 340 of 1406 records
Pulled 341 of 1406 records
Pulled 342 of 1406 records
Pulled 343 of 1406 records
P

Pulled 611 of 1406 records
Pulled 612 of 1406 records
Pulled 613 of 1406 records
Pulled 614 of 1406 records
Pulled 615 of 1406 records
Pulled 616 of 1406 records
Pulled 617 of 1406 records
Pulled 618 of 1406 records
Pulled 619 of 1406 records
Pulled 620 of 1406 records
Pulled 621 of 1406 records
Pulled 622 of 1406 records
Pulled 623 of 1406 records
Pulled 624 of 1406 records
Pulled 625 of 1406 records
Pulled 626 of 1406 records
Pulled 627 of 1406 records
Pulled 628 of 1406 records
Pulled 629 of 1406 records
Pulled 630 of 1406 records
Pulled 631 of 1406 records
Pulled 632 of 1406 records
Pulled 633 of 1406 records
Pulled 634 of 1406 records
Pulled 635 of 1406 records
Pulled 636 of 1406 records
Pulled 637 of 1406 records
Pulled 638 of 1406 records
Pulled 639 of 1406 records
Pulled 640 of 1406 records
Pulled 641 of 1406 records
Pulled 642 of 1406 records
Pulled 643 of 1406 records
Pulled 644 of 1406 records
Pulled 645 of 1406 records
Pulled 646 of 1406 records
Pulled 647 of 1406 records
P

Pulled 915 of 1406 records
Pulled 916 of 1406 records
Pulled 917 of 1406 records
Pulled 918 of 1406 records
Pulled 919 of 1406 records
Pulled 920 of 1406 records
Pulled 921 of 1406 records
Pulled 922 of 1406 records
Pulled 923 of 1406 records
Pulled 924 of 1406 records
Pulled 925 of 1406 records
Pulled 926 of 1406 records
Pulled 927 of 1406 records
Pulled 928 of 1406 records
Pulled 929 of 1406 records
Pulled 930 of 1406 records
Pulled 931 of 1406 records
Pulled 932 of 1406 records
Pulled 933 of 1406 records
Pulled 934 of 1406 records
Pulled 935 of 1406 records
Pulled 936 of 1406 records
Pulled 937 of 1406 records
Pulled 938 of 1406 records
Pulled 939 of 1406 records
Pulled 940 of 1406 records
Pulled 941 of 1406 records
Pulled 942 of 1406 records
Pulled 943 of 1406 records
Pulled 944 of 1406 records
Pulled 945 of 1406 records
Pulled 946 of 1406 records
Pulled 947 of 1406 records
Pulled 948 of 1406 records
Pulled 949 of 1406 records
Pulled 950 of 1406 records
Pulled 951 of 1406 records
P

Pulled 1211 of 1406 records
Pulled 1212 of 1406 records
Pulled 1213 of 1406 records
Pulled 1214 of 1406 records
Pulled 1215 of 1406 records
Pulled 1216 of 1406 records
Pulled 1217 of 1406 records
Pulled 1218 of 1406 records
Pulled 1219 of 1406 records
Pulled 1220 of 1406 records
Pulled 1221 of 1406 records
Pulled 1222 of 1406 records
Pulled 1223 of 1406 records
Pulled 1224 of 1406 records
Pulled 1225 of 1406 records
Pulled 1226 of 1406 records
Pulled 1227 of 1406 records
Pulled 1228 of 1406 records
Pulled 1229 of 1406 records
Pulled 1230 of 1406 records
Pulled 1231 of 1406 records
Pulled 1232 of 1406 records
Pulled 1233 of 1406 records
Pulled 1234 of 1406 records
Pulled 1235 of 1406 records
Pulled 1236 of 1406 records
Pulled 1237 of 1406 records
Pulled 1238 of 1406 records
Pulled 1239 of 1406 records
Pulled 1240 of 1406 records
Pulled 1241 of 1406 records
Pulled 1242 of 1406 records
Pulled 1243 of 1406 records
Pulled 1244 of 1406 records
Pulled 1245 of 1406 records
Pulled 1246 of 1406 

...

Pulled 1086 of 1727 records
Pulled 1087 of 1727 records
Pulled 1088 of 1727 records
Pulled 1089 of 1727 records
Pulled 1090 of 1727 records
Pulled 1091 of 1727 records
Pulled 1092 of 1727 records
Pulled 1093 of 1727 records
Pulled 1094 of 1727 records
Pulled 1095 of 1727 records
Pulled 1096 of 1727 records
Pulled 1097 of 1727 records
Pulled 1098 of 1727 records
Pulled 1099 of 1727 records
Pulled 1100 of 1727 records
Pulled 1101 of 1727 records
Pulled 1102 of 1727 records
Pulled 1103 of 1727 records
Pulled 1104 of 1727 records
Pulled 1105 of 1727 records
Pulled 1106 of 1727 records
Pulled 1107 of 1727 records
Pulled 1108 of 1727 records
Pulled 1109 of 1727 records
Pulled 1110 of 1727 records
Pulled 1111 of 1727 records
Pulled 1112 of 1727 records
Pulled 1113 of 1727 records
Pulled 1114 of 1727 records
Pulled 1115 of 1727 records
Pulled 1116 of 1727 records
Pulled 1117 of 1727 records
Pulled 1118 of 1727 records
Pulled 1119 of 1727 records
Pulled 1120 of 1727 records
Pulled 1121 of 1727 

Pulled 1379 of 1727 records
Pulled 1380 of 1727 records
Pulled 1381 of 1727 records
Pulled 1382 of 1727 records
Pulled 1383 of 1727 records
Pulled 1384 of 1727 records
Pulled 1385 of 1727 records
Pulled 1386 of 1727 records
Pulled 1387 of 1727 records
Pulled 1388 of 1727 records
Pulled 1389 of 1727 records
Pulled 1390 of 1727 records
Pulled 1391 of 1727 records
Pulled 1392 of 1727 records
Pulled 1393 of 1727 records
Pulled 1394 of 1727 records
Pulled 1395 of 1727 records
Pulled 1396 of 1727 records
Pulled 1397 of 1727 records
Pulled 1398 of 1727 records
Pulled 1399 of 1727 records
Pulled 1400 of 1727 records
Pulled 1401 of 1727 records
Pulled 1402 of 1727 records
Pulled 1403 of 1727 records
Pulled 1404 of 1727 records
Pulled 1405 of 1727 records
Pulled 1406 of 1727 records
Pulled 1407 of 1727 records
Pulled 1408 of 1727 records
Pulled 1409 of 1727 records
Pulled 1410 of 1727 records
Pulled 1411 of 1727 records
Pulled 1412 of 1727 records
Pulled 1413 of 1727 records
Pulled 1414 of 1727 

Pulled 1672 of 1727 records
Pulled 1673 of 1727 records
Pulled 1674 of 1727 records
Pulled 1675 of 1727 records
Pulled 1676 of 1727 records
Pulled 1677 of 1727 records
Pulled 1678 of 1727 records
Pulled 1679 of 1727 records
Pulled 1680 of 1727 records
Pulled 1681 of 1727 records
Pulled 1682 of 1727 records
Pulled 1683 of 1727 records
Pulled 1684 of 1727 records
Pulled 1685 of 1727 records
Pulled 1686 of 1727 records
Pulled 1687 of 1727 records
Pulled 1688 of 1727 records
Pulled 1689 of 1727 records
Pulled 1690 of 1727 records
Pulled 1691 of 1727 records
Pulled 1692 of 1727 records
Pulled 1693 of 1727 records
Pulled 1694 of 1727 records
Pulled 1695 of 1727 records
Pulled 1696 of 1727 records
Pulled 1697 of 1727 records
Pulled 1698 of 1727 records
Pulled 1699 of 1727 records
Pulled 1700 of 1727 records
Pulled 1701 of 1727 records
Pulled 1702 of 1727 records
Pulled 1703 of 1727 records
Pulled 1704 of 1727 records
Pulled 1705 of 1727 records
Pulled 1706 of 1727 records
Pulled 1707 of 1727 

Pulled 246 of 1112 records
Pulled 247 of 1112 records
Pulled 248 of 1112 records
Pulled 249 of 1112 records
Pulled 250 of 1112 records
Pulled 251 of 1112 records
Pulled 252 of 1112 records
Pulled 253 of 1112 records
Pulled 254 of 1112 records
Pulled 255 of 1112 records
Pulled 256 of 1112 records
Pulled 257 of 1112 records
Pulled 258 of 1112 records
Pulled 259 of 1112 records
Pulled 260 of 1112 records
Pulled 261 of 1112 records
Pulled 262 of 1112 records
Pulled 263 of 1112 records
Pulled 264 of 1112 records
Pulled 265 of 1112 records
Pulled 266 of 1112 records
Pulled 267 of 1112 records
Pulled 268 of 1112 records
Pulled 269 of 1112 records
Pulled 270 of 1112 records
Pulled 271 of 1112 records
Pulled 272 of 1112 records
Pulled 273 of 1112 records
Pulled 274 of 1112 records
Pulled 275 of 1112 records
Pulled 276 of 1112 records
Pulled 277 of 1112 records
Pulled 278 of 1112 records
Pulled 279 of 1112 records
Pulled 280 of 1112 records
Pulled 281 of 1112 records
Pulled 282 of 1112 records
P

Pulled 550 of 1112 records
Pulled 551 of 1112 records
Pulled 552 of 1112 records
Pulled 553 of 1112 records
Pulled 554 of 1112 records
Pulled 555 of 1112 records
Pulled 556 of 1112 records
Pulled 557 of 1112 records
Pulled 558 of 1112 records
Pulled 559 of 1112 records
Pulled 560 of 1112 records
Pulled 561 of 1112 records
Pulled 562 of 1112 records
Pulled 563 of 1112 records
Pulled 564 of 1112 records
Pulled 565 of 1112 records
Pulled 566 of 1112 records
Pulled 567 of 1112 records
Pulled 568 of 1112 records
Pulled 569 of 1112 records
Pulled 570 of 1112 records
Pulled 571 of 1112 records
Pulled 572 of 1112 records
Pulled 573 of 1112 records
Pulled 574 of 1112 records
Pulled 575 of 1112 records
Pulled 576 of 1112 records
Pulled 577 of 1112 records
Pulled 578 of 1112 records
Pulled 579 of 1112 records
Pulled 580 of 1112 records
Pulled 581 of 1112 records
Pulled 582 of 1112 records
Pulled 583 of 1112 records
Pulled 584 of 1112 records
Pulled 585 of 1112 records
Pulled 586 of 1112 records
P

In [19]:
cars_df.to_csv(path_or_buf=r"C:\Users\Leito\Documents\ML_cars_data_pulling\datasets\dataset2020-12-24.csv"
               ,sep= "\t"
               ,encoding="utf-8"
               ,index = False)

In [ ]:
url = ''